# Generation of Bottom-Trapped Internal Tides
Conversion from barotropic to baroclinic tide, linear calculation following [Falahat & Nycander, 2015]

* assume cartesian grid with constant spacing dx, dy
* uses exponential fit for the stratification, with corresponding approximate solutions for the vertical modes (and associated wavenumber)

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from mpl_toolkits.basemap import Basemap as bm
import scipy.interpolate as itp
import scipy.signal as sig
import scipy.stats as stats
import scipy.special as sp
from netCDF4 import Dataset
import time
from datetime import datetime

from SW_Density import SW_Density as rhop # temporary
#from comp_rho import rhop
from distance_sphere_matproof import dist_sphere_matproof
from convert_TPXO_to_ellipses import get_tpxo8_on_grid
from PyFVCOM import tidal_ellipse as ellipse

doverb = True

In [2]:
# --- data location ---
path_data = '/data0/project/vortex/lahaye/Tide_Conv/input_data/' #'./input_data/' #
path_data = '/net/krypton'+path_data     # if not on LOPS hub
path_alpha = '/net/alpha/exports/sciences/data/REFERENCE_DATA/'

# --- climato ---
clim = "lucky"
if clim == "lucky":
    cname = path_data+"lucky_ts_meanSODA_winter.nc"

# --- topography dataset --- 
topo = "lucky" # "tpxo" # "srtm"
if topo == 'lucky':
    file_topo = path_data+"lucky_corgrd.nc"
    varx, vary = 'lon_rho', 'lat_rho'
    varh = 'h'

# --- tide dataset --- 
tide = 'tpxo8' #'tpxo7' #

if tide == 'lucky':
    uname = path_data+'luckym2_frc.nc' 
elif tide == "tpxo7":
    uname = path_alpha+"TIDES/TPXO7.2/u_tpxo7.2.nc"
    gname = uname.replace("u","grid")
elif tide == 'tpxo8':
    uname = path_alpha+"TIDES/TPXO8/uv.k1_tpxo8_atlas_30c_v1.nc"
    gname = uname.replace("uv.k1","grid")
    
zmin       = -400              # min depth to compute Ef [m], below contin. shelf roughly
g          = 9.81              # gravity [m s-2]
K1         = 2*np.pi/23.93     # K1 tide frequency [rad/hour] 
Erad = 6371e3                  # Earth radius [m]

nmods = 1

In [3]:
# ------ get mean topo on the grid, will be used to get N2b --------
# load the entire grid (regional modelling purpose)
nc   = Dataset(file_topo,'r') # etopo2 and srtm30 files have the same structure
if topo == 'lucky':
    lon_h = nc.variables[varx][:].T
    lat_h = nc.variables[vary][:].T
    nlon_h, nlat_h = lon_h.shape
    h_t = -nc.variables[varh][:].T
    h_t[h_t>=-2.5] = 0     # land points   # "raw" topo (for FFT)
    h_ma = np.ma.masked_where(h_t==0,-h_t) # masked topo
    dx_h = 1/nc.variables['pm'][:].T
    dy_h = 1/nc.variables['pn'][:].T
    gang = nc.variables['angle'][:].T # grid angle (rad., between xi (origin) and E, pos. cc-wise)
nc.close()

dx, dy = dx_h.mean(),dy_h.mean()
dhdx, dhdy = np.gradient(h_t,dx,dy)
nx, ny = h_t.shape

fcor = 2.*(2*np.pi/24.)*np.sin(np.deg2rad(lat_h)) # coriolis, rad/hour

In [4]:
# ------ extract Tides ------------------------------

if tide == 'lucky':
    raise ValueError("not implemented")
    nc = Dataset(uname,'r')    # all this is old version
    # angle between major axis and east [rad] (beware sign)
    phi = nc.variables['tide_Cangle'][0,...].T
    # tide phase ()
    pha = nc.variables['tide_Cphase'][0,...].T
    # tidal current amplitude (major, minor axes)
    ue = nc.variables['tide_Cmax'][0,...].T
    ve = nc.variables['tide_Cmin'][0,...].T
    nc.close()
    ubt,uphi,vbt,vphi, _ = ellipse.ep2ap(ue,ve/ue,phi+np.rad2deg(phi),pha)  # INC+grang correct ?
    del phi, ue, ve
elif tide == 'tpxo7':
    raise ValueError("not implemented, need to modify get_tpxo7_on_grid before")
elif tide == 'tpxo8':
    ure,uim,vre,vim = get_tpxo8_on_grid([uname,gname],lon_h,lat_h,return_what="comp",grang=gang) 
    ue = ure + 1.j*uim
    ve = vre + 1.j*vim
    del ure, uim, vre, vim


/home/lahaye/working_on/Int_Tide_Conv/convert_TPXO_to_ellipses.py:187: RuntimeWarning: invalid value encountered in true_divide
  ure = nc.variables['uRe'][indxu,indyu]*1e-4/hu    # cm²/s to m/s
/home/lahaye/working_on/Int_Tide_Conv/convert_TPXO_to_ellipses.py:188: RuntimeWarning: invalid value encountered in true_divide
  vre = nc.variables['vRe'][indxv,indyv]*1e-4/hv
/home/lahaye/working_on/Int_Tide_Conv/convert_TPXO_to_ellipses.py:189: RuntimeWarning: invalid value encountered in true_divide
  uim = nc.variables['uIm'][indxu,indyu]*1e-4/hu
/home/lahaye/working_on/Int_Tide_Conv/convert_TPXO_to_ellipses.py:190: RuntimeWarning: invalid value encountered in true_divide
  vim = nc.variables['vIm'][indxv,indyv]*1e-4/hv


In [5]:
# ------ extract density profile, compute N2 ------------------
if clim == "lucky":
    nc = Dataset(cname,'r')
    T = nc.variables['temp_roms_avg'][:]
    S = nc.variables['salt_roms_avg'][:]
    zz = nc.variables['depth'][:]
    nz = zz.size


rho = np.sort(rhop(T,S)) #SW_Density(T,S) # sorting is cheating here
rho0 = rho.mean()
frho = itp.pchip(zz[::-1],rho[::-1],extrapolate=True)
N2_tmp = -(g/rho0)*frho.derivative()(zz)    # take routine
# temporary fixing:
if N2_tmp[-1]==0: N2_tmp[-1] = 1e-8    # this is a dirty fix
indneg, = np.where(N2_tmp<=0.)
for ii in indneg:
    N2_tmp[ii] = (N2_tmp[ii-1] + N2_tmp[ii+1])/2
fN2 = itp.pchip(zz[::-1],N2_tmp,extrapolate=True)

# fit exponential profile: N \approx N0*exp(bz)
slope,intercept,r_val,p_val,std_err = stats.linregress(zz,np.log(N2_tmp**0.5))
N0  = np.exp(intercept)
b   = 1./slope
N2b = np.sqrt(fN2(h_t))

if doverb:
    if indneg.size>0:
        print('had to resort stratif for {} values'.format(indneg.size))
    print('exponential interpolation for stratification: N0={0:.3e}, b={1:.1f}'.format(N0,b))

def cn_expN(n,h,b=b,N0=N0):
    """ 
    approximate (WKBJ) phase speed exponential fit for the stratification 
    of the form N=N0.exp(z/b) with z algebraic depth
    h is bottom depth (absolute value) 
    omega, f: tidal and Coriolis frequencies (rad/s)
    following StLaurent & Garrett 2002"""
    return N0*b*(1-np.exp(-b*h))/n/np.pi


loaded stratif from netCDF file
computed N(z)
exponential interpolation for stratification: N0=2.967e-03, b=1472.7


In [10]:
omega = K1

pn = np.full((nmods,nx,ny),np.nan+0.j*np.nan)
un = np.full((nmods,nx,ny),np.nan+0.j*np.nan)
vn = np.full((nmods,nx,ny),np.nan+0.j*np.nan)
indox, indoy = np.where(h_ma>-zmin)
trunc = 5.0 # truncating integral at finite distance

for imod in range(1,nmods+1):
    cn = cn_expN(imod,h_ma)
    kn = np.sqrt(fcor**2-omega**2)/cn/3600
    nsx = int(trunc/kn[indox,indoy].min()/dx)
    nsy = int(trunc/kn[indox,indoy].min()/dy)
    gx, gy = np.meshgrid(np.arange(nsx),np.arange(nsy))
    delx = dx*(gx-nsx//2)
    dely = dy*(gy-nsy//2)
    tmes, tmeb = time.clock(), time.time()
    px, py = np.full((nx,ny),np.nan+0.j*np.nan), np.full((nx,ny),np.nan+0.j*np.nan)
    for ix, iy in zip(indox,indoy):
        idx, idy = np.where(np.sqrt(delx**2+dely**2) < trunc/kn[ix,iy])
        indok = np.where( (idx>=0) & (idx<nx) & (idy>=0) & (idy<ny) )
        idx = idx[indok]
        idy = idy[indok]
        k1ox = sp.kn(1,kn[ix,iy]*np.sqrt(delx[idx,idy]**2 + dely[idx,idy]**2)) \
                            / np.sqrt(delx[idx,idy]**2 + dely[idx,idy]**2)
        pn[imod-1,ix,iy] = np.nansum( (ue[idx,idy]*delx[idx,idy] + ve[idx,idy]*dely[idx,idy])\
                                   *k1ox )
        px[ix,iy] = np.nansum( (ue[idx,idy]*dhdx[idx,idy] + ve[idx,idy]*dhdy[idx,idy])\
                                   *k1ox*delx[idx,idy] )
        py[ix,iy] = np.nansum( (ue[idx,idy]*dhdx[idx,idy] + ve[idx,idy]*dhdy[idx,idy])\
                                   *k1ox*dely[idx,idy] )
        print("ix, iy = {0},{1} ; timing = {2:.2f},{3:.2f}".format(ix, iy, time.clock()-tmes \
                , time.time()-tmeb), "size:",idx.size,idy.size)
        tmes, tmeb = time.clock(), time.time()
    coef = 1.j * kn[indox,indoy]**2 * np.sqrt(2*cn[indox,indoy] \
            * N2b[indox,indoy] * abs(fcor[indox,indoy]) / h_ma[indox,indoy])*dx*dy/2/np.pi
    pn[imod-1,indox,indoy] *= coef*(fcor[indox,indoy]**2/omega**2-1)
    un[imod-1,indox,indoy] = -coef*( fcor[indox,indoy]/omega**2*py[indox,indoy] \
                                     - 1.j/omega*px[indox,indoy] )
    vn[imod-1,indox,indoy] = coef*( fcor[indox,indoy]/omega**2*px[indox,indoy] \
                                    + 1.j/omega*py[indox,indoy] )                          
        
        

ix, iy = 0,0 ; timing = 0.79,0.79 size: 683929 683929
ix, iy = 0,1 ; timing = 0.73,0.74 size: 683929 683929
ix, iy = 0,2 ; timing = 0.74,0.74 size: 683929 683929
ix, iy = 0,3 ; timing = 0.73,0.74 size: 683929 683929
ix, iy = 0,4 ; timing = 0.73,0.73 size: 683929 683929
ix, iy = 0,5 ; timing = 0.78,0.78 size: 683929 683929
ix, iy = 0,6 ; timing = 0.74,0.74 size: 683929 683929
ix, iy = 0,7 ; timing = 0.74,0.74 size: 683929 683929
ix, iy = 0,8 ; timing = 0.73,0.73 size: 683929 683929
ix, iy = 0,9 ; timing = 0.74,0.74 size: 683929 683929
ix, iy = 0,10 ; timing = 0.75,0.75 size: 683929 683929
ix, iy = 0,11 ; timing = 0.75,0.76 size: 683929 683929
ix, iy = 0,12 ; timing = 0.74,0.74 size: 683929 683929
ix, iy = 0,13 ; timing = 0.73,0.73 size: 683929 683929
ix, iy = 0,14 ; timing = 0.73,0.73 size: 683929 683929
ix, iy = 0,15 ; timing = 0.73,0.73 size: 683929 683929
ix, iy = 0,16 ; timing = 0.77,0.77 size: 683929 683929
ix, iy = 0,17 ; timing = 0.75,0.75 size: 683929 683929
ix, iy = 0,18 ; timi

KeyboardInterrupt: 